# Benchmarking cities in MENA

In support of an upcoming Urban flagship report, the MENA team is looking for a series of zonal statistics:

- Nighttime Lights, Population, and built-area:  
  - Entire FUA  
  - Its associated urban center / “core”  
  - Associated “periphery”  

The unit of analysis is the Functional Urban Areas (FUAs) from the [UCDB Database](https://human-settlement.emergency.copernicus.eu/ghs_stat_ucdb2015mt_r2019a.php). For each FUA, we need to grab the associated urban periphary (lower threshold urban areas)

In [ ]:
import sys
import os
import itertools
import rasterio

import geopandas as gpd
import pandas as pd
import numpy as np

from scipy.spatial import cKDTree
from shapely.geometry import Point
from operator import itemgetter

sys.path.insert(0, "C:/WBG/Work/Code/GOSTrocks/src")

import GOSTrocks.rasterMisc as rMisc

In [ ]:
data_folder = r"C:\WBG\Work\data\URBAN"
ucdb_file = os.path.join(data_folder, "GHS_UCDB_GLOBE_R2024A.gpkg")
fua_file = os.path.join(data_folder, "GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg")
fua_peripheries = os.path.join(data_folder, "FUA_peripheries.gpkg")
oxford_extents = os.path.join(data_folder, "OE_FUA_SHAPEFILE", "OE_FUA_SHAPEFILE.shp")

flaring_locations_file = "https://thedocs.worldbank.org/en/doc/d01b4aebd8a10513c0e341de5e1f652e-0400072024/related/2012-2023-individual-flare-volume-estimates.xlsx?_gl=1*19fhic5*_gcl_au*MzM5MTcxNjUwLjE3MTg2NTk5ODU."

out_folder = r"C:\WBG\Work\Projects\MENA_Urban\RESULTS\ZONAL_RES\NTL"

In [ ]:
flaring_d = pd.read_excel(flaring_locations_file)
flaring_d["ID"] = flaring_d.index
flaring_geoms = [Point(x) for x in zip(flaring_d["Longitude"], flaring_d["Latitude"])]
flaring_d = gpd.GeoDataFrame(flaring_d, geometry=flaring_geoms, crs=4326)
all_flares = flaring_d.unary_union
flaring_d.head()

In [ ]:
inU = gpd.read_file(ucdb_file)
inF = gpd.read_file(fua_file)
m_crs = inF.crs
inF = inF.to_crs(inU.crs)
inO = gpd.read_file(oxford_extents)
inO = inO.to_crs(inU.crs)

inU["geometry"] = inU.buffer(0)
inF["geometry"] = inF.buffer(0)

# Adjust for flaring

Two steps to adjust the nighttime lights data for flaring:
1. Mute the nighttime lights data within a buffer zone of each flaring location
2. Determine distances between city extents (FUA) and nearest flaring location

In [ ]:
# 2 Distance calculations
def ckdnearest(gdfA, gdfB, gdfB_cols=["ID"]):
    A = np.concatenate([np.array(geom.coords) for geom in gdfA.geometry.to_list()])
    B = [np.array(geom.coords) for geom in gdfB.geometry.to_list()]
    B_ix = tuple(
        itertools.chain.from_iterable(
            [itertools.repeat(i, x) for i, x in enumerate(list(map(len, B)))]
        )
    )
    B = np.concatenate(B)
    ckd_tree = cKDTree(B)
    dist, idx = ckd_tree.query(A, k=1)
    idx = itemgetter(*idx)(B_ix)
    gdf = pd.concat(
        [
            gdfA,
            gdfB.loc[idx, gdfB_cols].reset_index(drop=True),
            pd.Series(dist, name="dist"),
        ],
        axis=1,
    )
    return gdf


inF_centroid = inF.copy()
inF_centroid["geometry"] = inF_centroid["geometry"].centroid
inF_centroid = inF_centroid.to_crs(m_crs)
flaring_d = flaring_d.to_crs(m_crs)

nearest_calc = ckdnearest(inF_centroid, flaring_d)
inF = pd.merge(inF, nearest_calc.loc[:, ["eFUA_ID", "dist"]], on="eFUA_ID")

In [ ]:
ntl_image = r"C:\WBG\Work\data\NTL\VNL_v21_npp_2019_global_vcmslcfg_c202205302300.average.dat.tif"  # ntl_images[0]
# 1 Mute nighttime lights data within a mask
### Buffer the flare locations by the defined distance
buffer_dist = 5000  # (metres)
buffered_flare = flaring_d.copy()
buffered_flare["geometry"] = buffered_flare["geometry"].apply(
    lambda x: x.buffer(buffer_dist)
)
buffered_flare = buffered_flare.to_crs(4326)

### create a mask raster using the buffered flare locations
# ntl_images = ntlMisc.aws_search_ntl()
flare_mask = rMisc.rasterizeDataFrame(
    buffered_flare, None, templateRaster=ntl_image, nodata=0
)
flare_mask = (~flare_mask["vals"].astype(bool)).astype(int)

In [ ]:
### Use the mask in the zonal calculation
finalF = inF.copy()  # inF.copy()
prefix = "FUA"
date = 2019  # os.path.basename(ntl_image).split("_")[2][:6]
raw_ntl = rMisc.zonalStats(finalF, ntl_image, minVal=0.1, reProj=True)
raw_ntl = pd.DataFrame(raw_ntl, columns=["SUM", "MIN", "MAX", "MEAN"])
finalF[f"raw_{date}"] = raw_ntl["SUM"]

ntl_r = rasterio.open(ntl_image)
ntl_data = ntl_r.read()
masked_ntl_data = ntl_data * flare_mask
with rMisc.create_rasterio_inmemory(
    ntl_r.profile, masked_ntl_data
) as masked_ntl_raster:
    masked_ntl = rMisc.zonalStats(finalF, masked_ntl_raster, minVal=0.1, reProj=True)
    masked_ntl = pd.DataFrame(masked_ntl, columns=["SUM", "MIN", "MAX", "MEAN"])
    finalF[f"mask_{date}"] = masked_ntl["SUM"]

finalF.to_file(
    f"{out_folder}/{prefix}_{date}_{buffer_dist}_zonal_ntl.gpkg", driver="GPKG"
)
pd.DataFrame(finalF.drop(columns="geometry")).to_csv(
    f"{out_folder}/{prefix}_{date}_{buffer_dist}_zonal_ntl.csv"
)

# Create mapping and debugging data

In [ ]:
# Write flare mask to disk
flare_mask_file = os.path.join("/home/wb411133/temp", "flare_mask.tif")
flare_profile = ntl_r.profile.copy()
flare_mask = flare_mask.astype("int16")
flare_profile.update(dtype=flare_mask.dtype)

with rasterio.open(flare_mask_file, "w", **flare_profile) as out_flare:
    out_flare.write_band(1, flare_mask)

In [ ]:
# Combine raw and masked ntl results with urban extents
outF = nearest_calc.copy()
outF["rawNTL"] = raw_ntl_df["SUM"]
outF["maskedNTL"] = masked_ntl_df["SUM"]
outF.to_file(
    os.path.join("/home/wb411133/temp", "urban_extents_ntl_flaring.gpkg"), driver="GPKG"
)

## Combine NTL results
We have zonal results for the entire FUA and for the core as monthly results; there is a two step process to create the final results:
1. Combine monthly results into annual results  
2. Use FUA and cores to generate three stats: FUA SoL, Core SoL, and Periphery SoL

In [ ]:
def combine_ntl_annual(curD):
    """curD is a data frame of ntl zonal results"""
    for yr in range(2012, 2024):
        cur_columns = [x for x in curD.columns if f"ntl_{yr}" in x]
        sel_d = curD.loc[:, cur_columns]
        curD[f"ntl{yr}_SoL"] = sel_d.sum(axis=1) / len(cur_columns)
    return curD


out_folder = "s3://wbg-geography01/URBANIZATION/MENA/ZONAL_RES/NTL/"
combine_ntl_annual(fua_zonal).to_csv(os.path.join(out_folder, "fua_VIIRS_SoL.csv"))
combine_ntl_annual(core_zonal).to_csv(os.path.join(out_folder, "core_VIIRS_SoL.csv"))

In [ ]:
combine_ntl_annual(core_zonal)

# DEBUGGING

In [ ]:
tempF = inF.loc[inF["eFUA_ID"] == 1281]
tempF["geometry"] = tempF["geometry"].iloc[0].buffer(0.001)

In [ ]:
rMisc.zonalStats(tempF, ntl_images[2], minVal=0.1, reProj=True, allTouched=True)

In [ ]:
tempF["geometry"].iloc[0].buffer(0.001)

In [ ]:
tempF